# Pedestrian flows in Newcastle City Centre

These graphs analyse the pedestrian count data obtained from a small number of CCTV cameras in the centre of Newcastle, processed in real-time using computer vision to count pedestrians that cross lines. The direction of travel is also obtained.

In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import pickle
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sys
import gc
import re
from IPython.display import display, HTML
from textwrap import wrap

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London'
})

In [ ]:
# Used across all of the plots
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())

colourUp = '#f64a8a'
colourDown = '#233067'

print('Last updated %s' % (datetime.datetime.now().strftime('%d %B %Y %H:%M')))

In [ ]:
# Load in baseline data that's obtained month-by-month in the baseline data script
peopleDataRaw = pickle.load(open('cache/baseline-pedestrian-flow-api-json.pkl', 'rb'))

In [ ]:
peopleDataRequestSince = 0
peopleDataBaselineStart = sys.maxsize

for sensor in peopleDataRaw:
    for variable in sensor['data']:
        for record in sensor['data'][variable]:
            if record['Timestamp'] > peopleDataRequestSince:
                peopleDataRequestSince = record['Timestamp']
            if record['Timestamp'] < peopleDataBaselineStart:
                peopleDataBaselineStart = record['Timestamp']
            
peopleDataRequestSince = datetime.datetime.fromtimestamp(peopleDataRequestSince / 1000)
peopleDataBaselineStart = datetime.datetime.fromtimestamp(peopleDataBaselineStart / 1000)

print('Using baseline profile data for each cross line from %s until %s' % (peopleDataBaselineStart, peopleDataRequestSince))

In [ ]:
# Add to the baseline data the most recent data
peopleRequestBase = 'https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/data/json/'
peopleRequestVariables = [
    'Walking North East',
    'Walking North West',
    'Walking South East',
    'Walking North',
    'Walking South',
    'Walking East',
    'Walking West'
]
peopleRequestIRI = ('%s?variable=%s&starttime=%s&endtime=%s') % (
    peopleRequestBase,
    ','.join(str(x).replace(' ', '%20') for x in peopleRequestVariables),
    (peopleDataRequestSince + pd.Timedelta(seconds=1)).strftime('%Y%m%d%H%M%S'),
    (dateToday + pd.Timedelta(days=1.5)).strftime('%Y%m%d%H%M%S')
)

#print('Loading recent data...')
peopleDataWindow = json.loads(
  urllib
    .request
    .urlopen(peopleRequestIRI)
    .read()
    .decode('utf-8')
)['sensors']

for sensor in peopleDataWindow:
    for variable in sensor['data']:
        targetSensor = next(s for s in peopleDataRaw if s['Sensor Name'] == sensor['Sensor Name'])

        if variable not in targetSensor['data']:
            continue

        targetVariable = targetSensor['data'][variable]

#        print('  Found %u recent observations and %u baseline observations for %s on %s' % (
#            len(sensor['data'][variable]),
#            len(targetSensor['data'][variable]),
#            variable,
#            targetSensor['Sensor Name']['0']
#        ))

        targetVariable.extend(sensor['data'][variable])

In [ ]:
#print('Obtained data from %u sensors.' % len(peopleDataRaw))

In [ ]:
# Number of seconds to resample the pedestrian data to for all subsequent processing
peopleCountInterval = 900

In [ ]:
cameraFriendlyNames = {
    #'PER_PEOPLE_BLACKETT-NORTHUMBERLAND-W': 'Blackett St pavement (north side) outside Rox',
    #'PER_PEOPLE_BLACKETT-BOOTS': 'Blackett St outside Boots',
    'PER_PEOPLE_THE_CORE_LINE_0': 'Blue Star Square at Newcastle Helix (east side)',
    'PER_PEOPLE_THE_CORE_LINE_1': 'Blue Star Square at Newcastle Helix (west side)',
    'PER_PEOPLE_USB_LINE_0': 'Science Square at Newcastle Helix',
    'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_0': 'Northumberland St near Fenwick (west side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_1': 'Northumberland St near Fenwick (east side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_MID_DISTANCE_HEAD_0': 'Northumberland St near TK Maxx',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_0': 'John Dobson St (west side) pavement near Goldsmiths',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_1': 'John Dobson St crossing island between Blackett St and New Bridge St West',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_2': 'John Dobson St (east side) pavement near The Stack',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_3': 'Pavement (south side) corner John Dobson St and Blackett St',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_4': 'Pavement (south side) corner John Dobson St and New Bridge St West',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_5': 'Blackett St crossing from John Dobson St to Northumberland St (west side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_6': 'New Bridge St West crossing John Dobson St to Northumberland St (east side)'
}

peopleCountFrames = {}

for sensor in peopleDataRaw:
    dfSensor = None
    
    if sensor['Sensor Name']['0'] not in cameraFriendlyNames:
        continue
        
    cameraName = cameraFriendlyNames[sensor['Sensor Name']['0']]
    
    for variable in sensor['data'].keys():
        # Skip vehicle counts or bus data
        if 'Walking' not in variable:
            continue
        
        # Ignore everything but the timestamp and the value
        dfPeopleTs = pd.DataFrame.from_records(sensor['data'][variable], columns=['Timestamp', 'Value'])
        
        # Timestamps are milliseconds since 1970 (epoch), so convert them to proper timestamps
        dfPeopleTs['Timestamp'] = (dfPeopleTs['Timestamp'].astype(int) / 1000).apply(datetime.datetime.fromtimestamp)
        
        dfPeopleTs = dfPeopleTs.rename(columns={'Value': variable})
        dfPeopleTs.set_index('Timestamp', inplace=True, drop=True)
        
        if dfSensor is None:
            dfSensor = dfPeopleTs
        else:
            dfSensor = dfSensor.join(dfPeopleTs)
    
    if dfSensor is None:
        #print('No data available from "%s" camera.' % cameraName)
        continue
    #else:
    #    print('Data from "%s" camera has been resampled to %u second intervals.' % (cameraName, peopleCountInterval))
    
    dfSensor = dfSensor.resample('%us' % peopleCountInterval).apply(lambda x: np.sum(x.values))
    peopleCountFrames[cameraName] = dfSensor
    
peopleDataRaw = None
gc.collect() ;

In [ ]:
# If you need to preview the data from one of the cameras...
#testCam = list(peopleCountFrames.keys())[0]
#print(testCam)
#peopleCountFrames[testCam]

In [ ]:
# Ignore non-numeric columns in the dataframe
plottableTypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
perMinuteFactor = (peopleCountInterval / 60)

## Summary

In [ ]:
sensorsToSummarise = {
    'Northumberland St near TK Maxx': 'Newcastle: City centre shopping district (Northumberland St near TK Maxx)',
    'Pavement (south side) corner John Dobson St and Blackett St': 'Newcastle: City centre edge of shopping district (corner of Blackett St and John Dobson St)'
}

def classifyTime(t):
    hour = int(t.strftime('%H'))
    if hour < 7:
        return 'Night (19:00 - 07:00)'
    elif hour < 10:
        return 'Morning peak (07:00 - 10:00)'
    elif hour < 16:
        return 'Intra-peak (10:00 - 16:00)'
    elif hour < 19:
        return 'Evening peak (16:00 - 19:00)'
    else:
        return 'Night (19:00 - 07:00)'

periodDurations = {
    'Morning peak (07:00 - 10:00)': 3,   # 7 - 10
    'Intra-peak (10:00 - 16:00)': 6,     # 10 - 16
    'Evening peak (16:00 - 19:00)': 3,   # 16 - 19
    'Night (19:00 - 07:00)': 12
}

allPeriods = list(periodDurations.keys())

for sensorName in sensorsToSummarise.keys():
    dateIndex = []
    directionIndex = []
    summaryData = []
    
    dfSensor = peopleCountFrames[sensorName].copy()
    dfSensor.insert(0, 'Date', dfSensor.index.to_series().apply(lambda t: t.date()))
    dfSensor.insert(0, 'Day of week', dfSensor.index.to_series().apply(lambda t: t.strftime('%A')))
    dfSensor.insert(1, 'Time of day', dfSensor.index.to_series().apply(lambda t: t.strftime('%H:%M:%S')))
    dfSensor.insert(1, 'Period', dfSensor.index.to_series().apply(classifyTime))

    dfDailyPeriodTotals = dfSensor.groupby(['Date', 'Day of week', 'Period'], as_index=False).sum()
    dfAveragePeriodTotals = dfDailyPeriodTotals.groupby(['Day of week', 'Period'], as_index=False).median()
    
    for d in range(9, -1, -1):
        sensorDate = dateToday - pd.Timedelta(days=d)
        dateIndex.extend(np.repeat(sensorDate.strftime('%A %d %B'), 2))

        directionColumns = dfSensor.select_dtypes(plottableTypes).columns
        directionIndex.extend(directionColumns)
                         
        dfSensorOnDate = dfSensor.loc \
            [(sensorDate <= dfSensor.index) &
            (dfSensor.index < sensorDate + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorLastWeek = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=7) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=7) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorYesterday = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=1) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=1) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]

        dfSensorAverageDayOfWeek = dfAveragePeriodTotals[dfAveragePeriodTotals['Day of week'] == sensorDate.strftime('%A')]

        for direction in directionColumns:
            summaryRow = []
            for period in allPeriods:
                periodStats = dfSensorOnDate[dfSensorOnDate.index == period][direction]
                if not periodStats['sum'].empty:

                    # Convert to an hourly value
                    periodTotal = periodStats['sum'].values[0]
                    periodHourly = periodTotal / periodStats['count'].values[0] * (3600 / peopleCountInterval)

                    # Change on yesterday
                    yesterdayHourly = dfSensorYesterday[dfSensorYesterday.index == period][direction]
                    yesterdayHourly = (yesterdayHourly['sum'].values[0] / yesterdayHourly['count'].values[0]) * (3600 / peopleCountInterval)
                    yesterdayChange = (periodHourly - yesterdayHourly) / yesterdayHourly

                    # Change on last week
                    lastWeekHourly = dfSensorLastWeek[dfSensorLastWeek.index == period][direction]
                    lastWeekHourly = (lastWeekHourly['sum'].values[0] / lastWeekHourly['count'].values[0]) * (3600 / peopleCountInterval)
                    lastWeekChange = (periodHourly - lastWeekHourly) / lastWeekHourly

                    # Change on normal profile
                    profileHourly = dfSensorAverageDayOfWeek[dfSensorAverageDayOfWeek['Period'] == period][direction]
                    profileHourly = profileHourly.values[0] / periodDurations[period]
                    profileChange = (periodHourly - profileHourly) / profileHourly

                    summaryRow.extend([
                        periodHourly, # Total
                        yesterdayChange, # Change on yesterday
                        lastWeekChange, # Change on last week
                        profileChange, # Change on average
                    ])
                else:
                    summaryRow.extend(np.repeat(0.0, 4))     
            summaryData.append(summaryRow)

    rowIndex = pd.MultiIndex.from_arrays([
            dateIndex,
            directionIndex
        ],
        names=['Date', 'Direction']
    )

    formattersSummary = {}
    colPeriods = []
    colStats = []
    for period in allPeriods:
        formattersSummary[(period, 'Hourly average flow')] = '{:,.0f}'
        formattersSummary[(period, 'Change from day before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from week before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from annual average (%)')] = '{:+,.0%}'
        colPeriods.extend(np.repeat(period, 4))
        colStats.extend([
            'Hourly average flow',
            'Change from day before (%)',
            'Change from week before (%)',
            'Change from annual average (%)'
        ])

    colIndex = pd.MultiIndex.from_arrays(
        [colPeriods, colStats],
        names=['Period', 'Statistic']
    )

    dfSummary = pd.DataFrame(summaryData, columns=colIndex, index=rowIndex)
    dfSummaryStyler = dfSummary.style \
        .format(formattersSummary) \
        .set_caption(sensorsToSummarise[sensorName]) \
        .set_table_styles(
            [dict(selector="th",props=[('text-align', 'center')]),
                dict(selector="tr:nth-child(2) th.col_heading",
                     props=[('vertical-align', 'bottom'),
                            ('writing-mode', 'vertical-rl'),
                            ]),
             dict(selector="caption",props=[('font-weight', 'bold'), ('font-size', '120%')])
            ]
        )

    periodBarColours = {
        'Morning peak (07:00 - 10:00)': '#FFA07A50',
        'Intra-peak (10:00 - 16:00)': '#EE1F5F50',
        'Evening peak (16:00 - 19:00)': '#FFA07A50',
        'Night (19:00 - 07:00)': '#A0FF7A50'
    }

    for period in allPeriods:
        dfSummaryStyler.background_gradient(
            subset=[(period, 'Change from annual average (%)')],
            vmin=-1.0,
            vmax=1.0,
            cmap='PiYG'
        )
        dfSummaryStyler.bar(subset=[(period, 'Hourly average flow')], color=periodBarColours[period], vmin=0)

    display(HTML(dfSummaryStyler._repr_html_()))

## Each camera during the last 28 days

In [ ]:
historicCutOffDays = 28

for sensorName in sorted(peopleCountFrames.keys()):
    dfSensor = peopleCountFrames[sensorName]
    dfSensor = dfSensor \
        [dfSensor.index >= dateToday - pd.Timedelta(days=historicCutOffDays)] \
        .copy()

    # We may have data from the API, but nothing in the last N days, so don't attempt
    # a graph
    if np.all(np.isnan(dfSensor)):
        continue

    # Show the last two days of data as a larger area, so the detail is visible
    axisBreak = datetime.datetime.now() - datetime.timedelta(days=2)

    # People per minute for 'walking' columns
    directionDataOld = [dfSensor[ : axisBreak][direction] / perMinuteFactor
        for direction in dfSensor.select_dtypes(plottableTypes).columns]
    directionDataRecent = [dfSensor[axisBreak : ][direction] / perMinuteFactor
        for direction in dfSensor.select_dtypes(plottableTypes).columns]
    directionLabels = [direction for direction in dfSensor.columns]

    fig = plt.figure(figsize=(18,6.5))
    gs = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[3, 1])
    ax = fig.add_subplot(gs[0, 0])
    axRecent = fig.add_subplot(gs[0, 1])

    fig.suptitle(sensorName)
    ax.set_xlabel('Date')
    axRecent.set_xlabel('Last 48 hours')
    ax.set_ylabel('Pedestrians per minute')

    for i in range(len(directionDataOld)):
        ax.stackplot(
          dfSensor[ : axisBreak].index, 
          np.zeros(len(directionDataOld[i])),
          directionDataOld[i] * (-1 if i % 2 == 0 else +1),
          colors=[colourUp] if i % 2 == 0 else [colourDown]
        )
        axRecent.stackplot(
          dfSensor[axisBreak : ].index, 
          np.zeros(len(directionDataRecent[i])),
          directionDataRecent[i] * (-1 if i % 2 == 0 else +1), 
          labels=[directionLabels[i]],
          colors=[colourUp] if i % 2 == 0 else [colourDown]
        )

    yMax = np.max(np.amax(dfSensor.select_dtypes(plottableTypes) / perMinuteFactor, axis=None))

    if np.isnan(yMax):
        continue

    ax.set_xlim([np.min(dfSensor.index), axisBreak])
    ax.set_ylim([-yMax, yMax])
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: abs(y)))
    axRecent.set_xlim([axisBreak, np.max(dfSensor.index)])
    axRecent.set_ylim([-yMax, yMax])
    axRecent.yaxis.tick_right()
    axRecent.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: abs(y)))

    dataFormatMajor = mdates.DateFormatter('%a %d %b')
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1, byweekday=mdates.MO))
    ax.xaxis.set_major_formatter(dataFormatMajor)
    ax.xaxis.set_tick_params(which='major', pad=15)
    ax.margins(x=0)
    axRecent.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    axRecent.xaxis.set_major_formatter(mdates.DateFormatter('%H\n%d'))
    axRecent.margins(x=0)

    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
    axRecent.xaxis.set_minor_locator(mdates.HourLocator(byhour=[0, 6, 12, 18]))
    axRecent.xaxis.set_minor_formatter(mdates.DateFormatter('%H'))

    # Highlight areas with missing data or zero people counts
    #dfGaps = np.any(dfSensor.resample('3600s').sum() == 0.0, axis=1)
    #ax.fill_between(dfGaps.index, 0, ax.get_ylim()[1], where=dfGaps,
    #            facecolor='grey', alpha=0.1)

    ax.spines['right'].set_visible(False)
    axRecent.spines['left'].set_visible(False)

    fig.subplots_adjust(wspace=0.04)
    axRecent.legend()
    
    plt.savefig(
        'output/pedestrian-flows_profile-28-days_%s.png' 
        % re.compile('[^a-z0-9]+').sub('-', sensorName.lower())
    )


## Each camera compared to normal daily profiles and averages

Some of the cameras aren't processed in real-time so are skipped in these graphs.

* The shaded area represents a normal percentile boundary obtained for that day of the week during the last year before the outbreak.
* The dotted line represents the median, so an average on that day of the week.
* The solid line represents the actual observed data.

In [ ]:
lowerPercentileRange = 15
upperPercentileRange = 85

def plotDateAgainstProfile(plotDate):
    sensorList = sorted(peopleCountFrames.keys())
    sensorWithDataToday = []

    for sensorName in sensorList:
        dfSensorToday = peopleCountFrames[sensorName][peopleCountFrames[sensorName].index >= plotDate]
        if len(dfSensorToday.index) > 0:
            sensorWithDataToday.append(sensorName)
    sensorCount = len(sensorWithDataToday)

    gridWidth = 3
    gridHeight = math.ceil(sensorCount / gridWidth)

    fig = plt.figure(figsize=(15, 1 + gridHeight * 4), constrained_layout=True)
    #fig.subplots_adjust(wspace=0.2, hspace=0.5)
    gs = fig.add_gridspec(ncols=gridWidth, nrows=gridHeight)

    subPlotIdx = 0
    for sensorName in sensorWithDataToday:
        dfSensor = peopleCountFrames[sensorName].copy()
        dfSensor.insert(0, 'Day of week', dfSensor.index.to_series().apply(lambda t: t.strftime('%A')))
        dfSensor.insert(1, 'Time of day', dfSensor.index.to_series().apply(lambda t: t.strftime('%H:%M:%S')))

        # Scale for per-minute values
        for c in dfSensor.select_dtypes(plottableTypes).columns:
            dfSensor[c] = dfSensor[c] / perMinuteFactor

        # Fetch today's data only, but not within the last N minutes (might be misleading if half
        # way through a bucket)
        dfSensorToday = dfSensor.loc \
          [(plotDate <= dfSensor.index) &
           (dfSensor.index <= plotDate + pd.Timedelta(hours=24)) &
          (dfSensor.index < np.max(dfSensor.index) - pd.Timedelta(seconds=peopleCountInterval))]

        # Calculate an average and quartiles for every 15 minutes on each day of the week
        aggregateColumns = ['Day of week', 'Time of day']
        dfDailyProfile = dfSensor.groupby(aggregateColumns, group_keys=False, as_index=False).median()
        dfDailyLQ = dfSensor.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(lowerPercentileRange / 100)
        dfDailyUQ = dfSensor.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(upperPercentileRange / 100)
        for direction in dfDailyProfile.select_dtypes(plottableTypes).columns:
            dfDailyProfile['%s: Lower percentile' % direction] = dfDailyLQ[direction]
            dfDailyProfile['%s: Upper percentile' % direction] = dfDailyUQ[direction]

        # Determine the maximum this graph will reach on ANY day of the week
        sensorWeeklyMax = dfSensor \
            .groupby(aggregateColumns, group_keys=False, as_index=False) \
            .quantile(0.85) \
            .copy() \
            .select_dtypes(plottableTypes) \
            .max(axis=0) \
            .max(axis=0)
            
        ax = fig.add_subplot(gs[math.floor(subPlotIdx / gridWidth), subPlotIdx % gridWidth])

        timeLocator = mdates.AutoDateLocator(minticks=4, maxticks=6)
        ax.set_title('\n'.join(wrap(sensorName, 40)), fontsize=12)
        ax.set_xlabel('Date')
        ax.xaxis.set_major_locator(timeLocator)
        ax.xaxis.set_major_formatter(
            mdates.ConciseDateFormatter(
                locator=timeLocator,
                offset_formats=['', '%Y', '%b-%Y', '%d-%b-%Y', '%d-%b-%Y', '%d-%b-%Y %H:%M']
            )
        )
        ax.set_ylabel('Pedestrians per minute')
        ax.set_ylim([0.0, sensorWeeklyMax])
        ax.set_xlim([plotDate, plotDate + pd.Timedelta(hours=24)])

        # Generate an average series on this specific day with full timestamps
        dfTodayProfile = dfDailyProfile[dfDailyProfile['Day of week'] == plotDate.strftime('%A')].copy()
        dfTodayProfile['Timestamp'] = dfDailyProfile['Time of day'].apply(
          lambda timeOfDay: dateutil.parser.parse(
            '%sT%sZ' % (plotDate.strftime('%Y-%m-%d'), timeOfDay)
          )
        )

        lineColour = colourUp
        plotPercentileRange = []
        plotMedian = []
        for direction in dfSensorToday.select_dtypes(plottableTypes).columns:
            plotPercentileRange.append(ax.fill_between(
                dfTodayProfile['Timestamp'],
                dfTodayProfile['%s: Lower percentile' % direction],
                dfTodayProfile['%s: Upper percentile' % direction],
                color=lineColour,
                linewidth=0,
                alpha=0.2
            ))
            plotMedian.append(ax.plot(
                dfTodayProfile['Timestamp'],
                dfTodayProfile[direction],
                color=lineColour,
                linestyle=':',
                alpha=0.4
            )[0])
            ax.plot(
                dfSensorToday.index,
                dfSensorToday[direction],
                color=lineColour,
                label=direction
            )
            lineColour = colourDown if lineColour == colourUp else colourUp

        ax.legend(loc=2, prop={'size': 9})
        subPlotIdx = subPlotIdx + 1

    plt.figtext(
        0.05,
        -0.03,
        'Urban Observatory (https://www.urbanobservatory.ac.uk/).\n'
        'Luke Smith <luke.smith@ncl.ac.uk>.',
        horizontalalignment='left',
        color='#606060',
        fontdict={'size': 11}
    )
    plt.figlegend(
        plotPercentileRange[:2] + plotMedian[:2],
        [
            '/',
            '%u to %u%%ile for %ss during last year' 
              % (lowerPercentileRange, upperPercentileRange, plotDate.strftime('%A')),
            '/',
            'Median for %s' % plotDate.strftime('%A')
        ],
        loc='lower right',
        ncol=4,
        labelspacing=0,
        handletextpad=0.4,
        columnspacing=0.4
    )

    #plt.tight_layout()
    
    plt.suptitle('%s' % plotDate.strftime('%A %d %B %Y'), fontsize='15', fontweight='bold')
    #plt.subplots_adjust(top=0.92)
    
    plt.savefig('output/pedestrian-flows_profile-comparison_%s.png' % plotDate.strftime('%d-%b-%Y'), bbox_inches='tight')
    plt.show()

### Today

In [ ]:
# Today
plotDateAgainstProfile(dateToday)

### Yesterday

In [ ]:
# Yesterday
plotDateAgainstProfile(dateToday - pd.Timedelta(hours=24))

In [ ]:
# Last two weeks
#for d in range(14, 1, -1):
#    plotDateAgainstProfile(dateToday - pd.Timedelta(days=d))